# `flarestack` Test Minimization Notebook

## 1. Directory Setup

In [ ]:
import logging
logging.basicConfig(level='INFO')

In [ ]:
import os
os.environ['FLARESTACK_SCRATCH_DIR']

In [ ]:
from flarestack.shared import host_server
from flarestack.data.icecube.ic_season import icecube_dataset_dir
print(f'Running at {host_server}, data directory is {icecube_dataset_dir}')

In [ ]:
from flarestack.shared import fs_scratch_dir
print(f'Scratch directory is {fs_scratch_dir}')

## 2. Flarestack Classes

Classes used in $\texttt{flarestack}$'s core functionality (e.g. `flarestack.core.energy_pdf.EnergyPDF`, `flarestack.core.minimisation.MinimisationHandler`, etc) have a class attribute `<class>.subclasses`.  
This is a dictionary with the structure `{<subclass name>: <subclass>}`.  

In [ ]:
from flarestack.core.minimisation import MinimisationHandler
MinimisationHandler.subclasses

For analyses we only have to pass a dictionary of the subclass names and corresponding parameters.  
To execute use `flarestack.cluster.submitter.Submitter`. This always works locally. For using the cluster, again, if you are running at DESY or WIPAC, you do not have to worry. We got you covered.

In [ ]:
from flarestack.cluster.submitter import Submitter
Submitter.submitter_dict

## 3. Example: Point Source Sensitivity

Let's try to calculate the 10-year point source sensitivity for our test catalogue.  
The input directory (with the analysis dictionaries), the output directory (plots, p-values, etc) and the cache directory (saved trials, etc) will be created accordingly.   
First we have to specify a name for the analysis.

In [ ]:
from flarestack.shared import plot_output_dir, name_pickle_output_dir
from glob import glob

In [ ]:
min_types = ('fixed', 'fit', 'mcmc', 'hmc')

def name_func(n_sources, gamma, min_type, scale, etc=None):
    """Creates name for output analysis files.
    
    :param n_sources: Number of sources in catalog
    :type n_sources: int
    
    :param gamma: Spectral index
    :type n_sources: float
    
    :param min_type: Minimization method (see MinimisationHandler.subclasses)
    :type n_sources: str
    
    :param etc: Additional simulation/run info
    :type n_sources: str
    
    :return path: Path used as name
    :rtype path: str
    """
    run_no = 1
    
    if min_type not in min_types:
        raise ValueError(f'Provide valid minimizer: {min_types}')
        
    path = f'analyses/{n_sources}source_gamma{gamma}_{min_type}_{scale}'
    
    if etc is not None:
        path += f'_{etc:s}'
        
    path += f'_run{run_no}'
    
    path_exist = os.path.exists(plot_output_dir(path)) or os.path.exists(name_pickle_output_dir(path))
    
    if path_exist:
        # Automatically covers cases where run_no == (n_sources or gamma)
        glob_path = path.split(f'_run{run_no}')[0]
        # Get all runs with same path
        previous_runs = glob(f'{name_pickle_output_dir(glob_path)}*')
        print(path, previous_runs)
        print(glob_path)
        # Get run numbers for previous runs, convert strings to ints
        run_nums = [int(i.split('_run')[1]) for i in previous_runs]
        # Sort run numbers
        run_nums.sort()
        # Get last run number, increase index by 1
        run_no = run_nums[-1] + 1
        path = f'{glob_path}_run{run_no}'
    else:
        # Path DNE, unchanged path (run 1)
        pass
    
    return path

In [ ]:
name = name_func(n_sources=5, gamma=2.0, min_type='fit', scale='sumscale',
                 etc='trials100_2012northertrack')
name

Our plot output directories will be:

In [ ]:
from flarestack.shared import plot_output_dir, name_pickle_output_dir
plot_output_dir(name), name_pickle_output_dir(name)

Public 3-year point source data.

In [ ]:
# from flarestack.data.icecube import ps_v003_p02
from flarestack.data.public import icecube_ps_3_year

icecube_ps_3_year.items()

IceCube northern tracks data.

In [ ]:
from flarestack.data.icecube.northern_tracks import nt_v002_p05

nt_v002_p05.nt_v002_p05.items()

In [ ]:
# from flarestack.data.icecube.ic_season import IceCubeDataset, icecube_dataset_dir
# from flarestack.data.icecube.northern_tracks import NTSeason, get_diffuse_binning

# nt_data_dir = icecube_dataset_dir + "northern_tracks/version-002-p05/"

# nt_v002_p05 = IceCubeDataset()

# sample_name = "northern_tracks_v002_p05"

# def generate_diffuse_season(name):
#     season = NTSeason(
#         season_name=name,
#         sample_name=sample_name,
#         exp_path=nt_data_dir + "dataset_8yr_fit_{0}_exp_compressed.npy".format(name),
#         mc_path=nt_data_dir + "dataset_8yr_fit_{0}_MC_compressed.npy".format(name),
#         grl_path=nt_data_dir
#         + "GRL/dataset_8yr_fit_{0}_exp_compressed.npy".format(name),
#         sin_dec_bins=get_diffuse_binning(name)[0],
#         log_e_bins=get_diffuse_binning(name)[1],
#     )
#     nt_v002_p05.add_season(season)
    
# seasons = ["IC59", "IC79", "IC86_2011", "IC86_2012_16"]

# for season in seasons:
#     generate_diffuse_season(season)
    
# nt_v002_p05.seasons

We want to inject a steady neutrino signal with a power law spectrum with $\gamma=2.5$. For other Energy or Time PDFs check `flarestack.core.energy_pdf` and `flarestack.core.time_pdf`.   \
This is as straight forward as:

In [ ]:
injection_energy = {
    "energy_pdf_name": "power_law",
    "gamma": 2.0
}

injection_time = {
    "time_pdf_name": "steady"
}

inj_kwargs = {
    "injection_energy_pdf": injection_energy,
    "injection_sig_time_pdf": injection_time
}

We are looking for a steady signal with a power law spectrum. 
We assume the background to be constant in time.  
We want to use the "standard" point source likelihood. More likelihood implementations in `flarestack.core.llh`

In [ ]:
llh_time = {
    "time_pdf_name": "steady"
}

llh_energy = {
    "energy_pdf_name": "power_law",
}

llh_time_bkg = {
    "time_pdf_name": "steady"
}

llh_kwargs = {
    "llh_name": "standard",
    "llh_energy_pdf": llh_energy,
    "llh_sig_time_pdf": llh_time,
    "llh_bkg_time_pdf": llh_time_bkg
}

We need a source catalogue. This catalogue will be a numpy array stored as a `.npy` file and we only pass the filename.   
For point sources the is a uitility function to generate dummy sources.

In [ ]:
# from flarestack.utils.prepare_catalogue import ps_catalogue_name
import numpy as np

sindec = 0.5
catalogue_path = "/Users/thomasahrens/Desktop/IceCube/sn-search/catalog/test_catalogue_5.npy"
print(f'your catalogue is located at {catalogue_path}')
catalogue = np.load(catalogue_path)
catalogue

Now we make a guess for our sensitivity.   
Note: $\texttt{flarestack}$ is using its own scale factor $k$.

In [ ]:
from flarestack.shared import flux_to_k, k_to_flux
flux_to_k(1), flux_to_k(1e-9)

Here we know where the sensitivity should be. Because the analysis has been done before.

In [ ]:
logging.basicConfig(level='ERROR')
from flarestack.icecube_utils.reference_sensitivity import reference_sensitivity
scale = flux_to_k(reference_sensitivity(np.sin(catalogue['dec_rad']))) * 3
scale

Now we just have to put all the info into one dictionary to pass to the `MinimisationHandler`

In [ ]:
mh_dict = {
    "name": name,                                           # unique name for the analysis
    "mh_name": "fit_weights",                               # name of the MinimisationHandler subcalss
    "dataset": icecube_ps_3_year.get_seasons(),             # the neutrino dataset
    "catalogue": catalogue_path,                            # path to the .npy catalogue file
    "inj_dict": inj_kwargs,                                 # info for the Injector
    "llh_dict": llh_kwargs,                                 # info for the LLH
    "scale": np.sum(scale),                                 # a guess for the sensitivity scale
    "n_trials": 100,                                        # number of trials to run (background trials will be run ten times this number!)
    "n_steps": 10,                                          # number of steps when injecting signal
    "allow_extrapolated_sensitivity": True                  # allow extrapolation in the sensitivity calculation (here we do because we only run very few trials)
}

To execute the analysis we defined above we create a submitter instance

In [ ]:
submitter = Submitter.get_submitter(
    mh_dict=mh_dict,                         # the analysis info
    use_cluster=False,                       # run it on the cluster if True
    n_cpu=7,                                # number of LOCAL CPUs to use, NOTE: the number of cluster CPUs has to be specified in the cluster_kwargs!
    do_sensitivity_scale_estimation=False,   # make a guess of the sensitivity scale, for options check flarestack.cluster.submitter
    remove_old_results=True,                 # if you are running the analysis again and something changed, maybe you want to remove old trials?
#   **cluster_kwargs                         # keyword arguments used when running the cluster, This depends on the cluster obviously
)

print(submitter)

Energise ......

In [ ]:
# %prun -T prun.txt submitter.analyse()
submitter.analyse()

To get the results we use the `ResultsHandler()`. This will also create some plots like the sensitivity fit, bias plots, etc. in the plot directory. If `OverfluctuationError`, set `do_sens=False` and `do_disc=False` in `ResultsHandler()` object.

```do_sens=False, do_disc=False```

In [ ]:
from flarestack.core.results import ResultsHandler
results_handler = ResultsHandler(submitter.mh_dict)
# results_handler.__dict__

In [ ]:
# Move "prun.txt" profiler file to output dir
# os.rename("prun.txt", os.path.join(plot_output_dir(name), "prun.txt"))

In [ ]:
print(fr'sensitivity flux: {results_handler.sensitivity:.2e} +{results_handler.sensitivity_err[1]}  -{results_handler.sensitivity_err[0]}')
print(f'reference: {reference_sensitivity(sindec)[0]}')
print(fr'sensitivity n_s: {results_handler.sensitivity * results_handler.flux_to_ns:.2e} +{results_handler.sensitivity_err[1] * results_handler.flux_to_ns}  -{results_handler.sensitivity_err[0] * results_handler.flux_to_ns}')

### MCMC seed values

In [ ]:
from glob import glob
import pickle

In [ ]:
def means_and_dev(name):
    """Calculates mean and standard deviation from `fit_weights` minimizer
    to then be input into `fit_weights_mcmc` minimizer.
    
    :param name: Analysis run name
    :type name: str
    
    :return mu: List of average parameter (n_s, gamma) values
    :rtype mu: list
    
    :return std: List of parameter (n_s, gamma) standard deviations
    :rtype std: list
    """
    path_to_pickles = os.path.join(name_pickle_output_dir(name), 'merged')
    pickles = glob(os.path.join(path_to_pickles, '*.pkl'))
    key_arrays = {}
    
    for pkl in pickles:
        pickle_path = pkl

        with open(pickle_path, 'rb') as file:
            pickle_data = pickle.load(file)

        if not key_arrays:
            key_arrays = {key:[] for key in pickle_data['Parameters'].keys()}

        for key, data in pickle_data['Parameters'].items():
                key_arrays[key].append(data)
        
    mu = []
    std = []

    for key, data in key_arrays.items():
        key_arrays[key] = np.array(sum(key_arrays[key], []))
        mu.append(float(f'{np.mean(key_arrays[key]):0.4f}'))
        std.append(float(f'{np.std(key_arrays[key]):0.4f}'))

    print(f"mu = {mu}")
    print(f"std = {std}")

In [ ]:
means_and_dev(name)

## 4. MCMC Analysis Plots

In [ ]:
import pickle
import corner
import matplotlib.pyplot as plt

In [ ]:
# Path to MCMC pickle directory
mcmc_pickle_path = os.path.join(name_pickle_output_dir(name), 'chains.pkl')

with open(mcmc_pickle_path, 'rb') as file:
    mcmc_pickle = pickle.load(file)
    
mcmc_pickle.shape

In [ ]:
def labels(name):
    corner_labels = []
    for source in range(len(catalogue)):
        corner_label = catalogue[source]['source_name'].decode()
        corner_labels.append('n_s: ' + label)
    corner_labels.append('gamma')
    
    return corner_labels

### Corner Plot

In [ ]:
def corner_plot(name, burn_in=0, save_fig=False, **kwargs):
    mcmc_pickle_path = os.path.join(name_pickle_output_dir(name), 'chains.pkl')

    with open(mcmc_pickle_path, 'rb') as file:
        mcmc_pickle = pickle.load(file)
        
    corner_labels = labels(name)

    truths = np.append(scale / 3, injection_energy['gamma'])
    
    reshaped_steps = mcmc_pickle[burn_in:].reshape((-1,ndim))
    
    fig = corner.corner(reshaped_steps, bins=30,
                        labels=corner_labels,
                        quantiles=[0.16, 0.5, 0.84],
                        truths=truths,
                        use_math_text=True,
                        show_titles=True, 
                        title_kwargs={"fontsize": 10},
                        plot_datapoints=False, 
                        **kwargs)
    
    if save_fig:
        plt.savefig(os.path.join(plot_output_dir(name), 'corner.png'))

In [ ]:
# Corner plot with burn in
corner_plot(name)

In [ ]:
# Corner plot without brun in, save figure
# corner_plot(name, burn_in=2000, save_fig=True)

### Walker Steps

In [ ]:
def walker_plot(name, n_steps, save_fig=False):
    ndim = len(catalogue) + 1
    fig, axes = plt.subplots(ndim, figsize=(15, 8), sharex=True)
    # samples = sampler.get_chain()
    walker_labels = labels(name)
    for i in range(ndim):
        ax = axes[i]
        ax.plot(mcmc_pickle[:, :, i], "k", alpha=0.1)
        ax.set_xlim(0, len(mcmc_pickle[:n_steps]))
        ax.set_ylabel(walker_labels[i], rotation=0, ha='right')
        ax.yaxis.set_label_coords(-0.05, 0.5)

    axes[-1].set_xlabel("step number")
    fig.tight_layout
    
    if save_fig:
        plt.savefig(os.path.join(plot_output_dir(name), 'walkers.png'))

In [ ]:
# Plot walkers, save figure 
walker_plot(name, n_steps=len(mcmc_pickle), save_fig=True)

In [ ]:
# Plot first 1000 steps
walker_plot(name, n_steps=1000)

### Autocorrelation Function

In [ ]:
import emcee.autocorr as eac

In [ ]:
eac.function_1d(mcmc_pickle[:,0,2])

In [ ]:
fig, axes = plt.subplots(ndim, figsize=(15, 8), sharex=True)
# samples = sampler.get_chain()
labels = corner_labels
for i in range(ndim):
    ax = axes[i]
    ax.plot(eac.function_1d(mcmc_pickle[:,0,i]), "k", alpha=0.1)
    ax.axvline(3 * eac.integrated_time(mcmc_pickle[:,0,:], quiet=True)[0], 0, 1)
#     ax.set_xlim(0, len(mcmc_pickle[:500]))
    ax.set_ylabel(labels[i], rotation=0, ha='right')
    ax.yaxis.set_label_coords(-0.05, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
for i in range(ndim):
    q_16, q_50, q_84 = corner.quantile(no_burn[:,i], [0.16, 0.5, 0.84]) # your x is q_50
    dx_down, dx_up = q_50-q_16, q_84-q_50
    print(f'{i:>3d} {corner_labels[i]:>40s} : {q_50:>7.2f} [{dx_down:<4.2f}, {dx_up:<4.2f}]')

In [ ]:
nsteps, nwalkers, nparams = mcmc_pickle.shape
acf = np.zeros(shape=(nsteps, nparams))
for i in range(nparams):
    temp = np.zeros(shape=(nsteps, nwalkers))
    for x in range(nwalkers):
        temp[:,x] = eac.function_1d(mcmc_pickle[:,x,i])
    acf[:,i] = temp.mean(axis=1)
    
acf

In [ ]:
# First n steps
n = 700
fig, axes = plt.subplots(ndim, figsize=(15, 8), sharex=True)
# samples = sampler.get_chain()
labels = corner_labels
act = eac.integrated_time(mcmc_pickle[:,:,:], quiet=True)
for i in range(ndim):
    ax = axes[i]
    ax.plot(acf[:,i], "k", alpha=0.5)
    ax.axvline(act[i], 0, 1)
    ax.set_xlim(0, len(mcmc_pickle[:n]))
    ax.set_ylabel(labels[i], rotation=0, ha='right')
    ax.yaxis.set_label_coords(-0.05, 0.5)

axes[-1].set_xlabel("step number");